### 1. 개발 환경 세팅

In [1]:
### 개발환경 세팅하기

# ▶ 한글 폰트 다운로드
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 10.3 MB of archives.
After this operation, 34.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum all 20200506-1 [10.3 MB]
Fetched 10.3 MB in 1s (11.6 MB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 123630 files and direc

In [2]:
# ▶ 한글 폰트 설정하기
import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic')
plt.rcParams['axes.unicode_minus'] =False

# ▶ Warnings 제거
import warnings
warnings.filterwarnings('ignore')

# ▶ Google drive mount or 폴더 클릭 후 구글드라이브 연결
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# 필요한 라이브러리 불러오기
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

### 2. 서울시 버스 정류장 데이터 전처리 및 EDA

In [4]:
import os
print(os.listdir('/content/'))

['.config', '서울시_정류장마스터_정보.csv', 'drive', '국내_지역별_관광명소데이터.csv', 'sample_data']


In [5]:
bus_loc = pd.read_csv('./서울시_정류장마스터_정보.csv', encoding="cp949")
bus_loc

,정류장_ID,정류장_명칭,정류장_유형,정류장_번호,위도,경도,버스도착정보안내기_설치_여부
0,100000001,종로2가사거리,중앙차로,1001,37.569806,126.987752,설치
1,100000002,창경궁.서울대학교병원,중앙차로,1002,37.579433,126.996521,설치
2,100000003,명륜3가.성대입구,중앙차로,1003,37.582580,126.998251,설치
3,100000004,종로2가.삼일교,중앙차로,1004,37.568579,126.987613,설치
4,100000005,혜화동로터리.여운형활동터,중앙차로,1005,37.586243,127.001744,설치
...,...,...,...,...,...,...,...
11329,129000043,김포IC(가상),가상정류장,41613,37.591442,126.766228,미설치
11330,129000044,양주IC(가상),가상정류장,68604,37.680124,126.919246,미설치
11331,129000045,양주IC(가상),가상정류장,68605,37.678402,126.916507,미설치
11332,129000110,난지도하수처리장(가상),가상정류장,35679,37.584193,126.848843,미설치


In [6]:
bus_loc['정류장_유형'].unique()

array(['중앙차로', '일반차로', '가로변시간', '가로변전일', '가상정류장', '마을버스'], dtype=object)

In [7]:
# 가상정류장은 의미 없으니까 제외
seoul_bus = bus_loc[bus_loc['정류장_유형'] != '가상정류장']
seoul_bus['정류장_유형'].unique()

array(['중앙차로', '일반차로', '가로변시간', '가로변전일', '마을버스'], dtype=object)

In [8]:
seoul_bus.isna().sum()

,0
정류장_ID,0
정류장_명칭,0
정류장_유형,0
정류장_번호,0
위도,0
경도,0
버스도착정보안내기_설치_여부,0


In [9]:
seoul_bus.duplicated(subset='정류장_ID').sum()

0

In [ ]:
# 서울의 버스 정류장 지도로 시각화

!pip install pandas folium

import pandas as pd
import folium

# 지도 초기화 (중심 좌표 설정)
# 중심 위치를 데이터의 첫 번째 위치로 설정
center_lat = seoul_bus['위도'].mean()
center_lon = seoul_bus['경도'].mean()
m = folium.Map(location=[center_lat, center_lon], zoom_start=12)

# 각 버스 노션을 지도에 추가
for idx, row in seoul_bus.iterrows():
    folium.Marker(
        location=[row['위도'], row['경도']],
        popup=row['정류장_명칭'],  # 마커를 클릭하면 나오는 정보
    ).add_to(m)

# 지도를 HTML 파일로 저장
m.save("seoul_bus_map.html")

# 완료 메시지
print("버스 노션 지도가 'seoul_bus_map.html' 파일로 저장되었습니다.")


버스 노션 지도가 'seoul_bus_map.html' 파일로 저장되었습니다.


In [ ]:
!pip install geopy

### 3. 관광지 데이터 전처리 및 EDA

In [ ]:
place_loc = pd.read_csv('./전국관광지정보표준데이터.csv', encoding='cp949')
place_loc = place_loc[place_loc['소재지도로명주소'].str.contains('서울', na=False)]
place_loc['관광지명'].unique

<bound method Series.unique of 4               서울 원효로 예수성심당
6                심원정 왜명강화지처비
7                    청암동 부군당
8                    산천동 부군당
9      3.1운동 용산인쇄소 노동자 만세시위지
               ...          
582                     허가바위
675             후암동 조선은행 사택지
824                 당고개 순교성지
825         옛 풍국제과 공장(현 오리온)
826                    용산신학교
Name: 관광지명, Length: 61, dtype: object>

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)     # 모든 행 출력
print(place_loc)

                          관광지명 관광지구분                  소재지도로명주소  \
4                 서울 원효로 예수성심당   관광지       서울특별시 용산구 원효로19길 49   
6                  심원정 왜명강화지처비   관광지       서울특별시 용산구 효창원로8길 28   
7                      청암동 부군당   관광지       서울특별시 용산구 원효로 17-11   
8                      산천동 부군당   관광지       서울특별시 용산구 효창원로15길 7   
9        3.1운동 용산인쇄소 노동자 만세시위지   관광지       서울특별시 용산구 원효로41길 33   
10                   이봉창 의사 집터   관광지         서울특별시 용산구 백범로 275   
11                     백범김구기념관   관광지          서울특별시 용산구 임정로 26   
12                이봉창 의사 역사울림관   관광지       서울특별시 용산구 백범로 281-9   
61                     남이장군 사당   관광지      서울특별시 용산구 효창원로 88-10   
62                     연복사탑중창비   관광지      서울특별시 용산구 한강대로21나길 7   
63   서울 구 용산철도병원 본관(현 용산역사박물관)   관광지   서울특별시 용산구 한강대로14길 35-29   
64                    왜고개 순교성지   관광지      서울특별시 용산구 한강대로40길 46   
65                옛 간조 경성지점 사옥   관광지      서울특별시 용산구 한강대로42길 13   
66                         새남터   관광지          서울특별시 용산구 이촌로 71   
67        

In [ ]:
seoul_place = place_loc[['관광지명', '소재지도로명주소', '위도', '경도']]

In [ ]:
seoul_place

,관광지명,소재지도로명주소,위도,경도
4,서울 원효로 예수성심당,서울특별시 용산구 원효로19길 49,37.534209,126.954593
6,심원정 왜명강화지처비,서울특별시 용산구 효창원로8길 28,37.533425,126.951860
7,청암동 부군당,서울특별시 용산구 원효로 17-11,37.534803,126.946373
8,산천동 부군당,서울특별시 용산구 효창원로15길 7,37.535662,126.953170
9,3.1운동 용산인쇄소 노동자 만세시위지,서울특별시 용산구 원효로41길 33,37.535439,126.958416
10,이봉창 의사 집터,서울특별시 용산구 백범로 275,37.540313,126.959653
11,백범김구기념관,서울특별시 용산구 임정로 26,37.544278,126.959253
12,이봉창 의사 역사울림관,서울특별시 용산구 백범로 281-9,37.540103,126.960852
61,남이장군 사당,서울특별시 용산구 효창원로 88-10,37.536475,126.958433
62,연복사탑중창비,서울특별시 용산구 한강대로21나길 7,37.529772,126.964742


In [10]:
# 관광지 데이터 더 괜찮은 거 찾음
# LC_LO: 경도, LC_LA: 위도, CTPRVN_NM: 시도명

tour_df = pd.read_csv('./국내_지역별_관광명소데이터.csv', encoding='utf-8')
tour_df

,ID,LCLAS_NM,MLSFC_NM,POI_ID,POI_NM,BHF_NM,ASSTN_NM,CL_CD,CL_NM,PNU,...,RDNMADR_CD,RDNMADR_NM,BULD_NO,LC_LO,LC_LA,GID_CD,LAST_CHG_DE,ORIGIN_NM,FILE_NM,BASE_DE
0,KCLANPO23N000000001,장소,관광지,76838,간절곶등대,NaN,NaN,60513,일반관광지,3171031025100280001,...,3.171043e+11,간절곶1길,39-2,129.360675,35.359029,마마690086,20240108,KT,KC_495_LLR_ATRCTN_2023,20231220
1,KCLANPO23N000000002,장소,관광지,141502,우성항공여행사,NaN,NaN,60403,항공사/여행사,4579025021102250000,...,4.579033e+11,중앙로,232,126.700823,35.434790,다마274157,20240108,KT,KC_495_LLR_ATRCTN_2023,20231220
2,KCLANPO23N000000003,장소,관광지,142394,우정여행사,NaN,NaN,60403,항공사/여행사,1114010300100310023,...,1.114041e+11,세종대로20길,15,126.978339,37.567237,다사539521,20240108,KT,KC_495_LLR_ATRCTN_2023,20231220
3,KCLANPO23N000000004,장소,관광지,145432,비산항공여행사,NaN,NaN,60403,항공사/여행사,1165010700100520006,...,1.165031e+11,서초중앙로,225,127.011566,37.501664,다사568448,20240108,KT,KC_495_LLR_ATRCTN_2023,20231220
4,KCLANPO23N000000005,장소,관광지,149376,부래여행사,NaN,NaN,60403,항공사/여행사,2611010700100550001,...,2.611031e+11,해관로,76,129.035622,35.105620,마라399800,20240108,KT,KC_495_LLR_ATRCTN_2023,20231220
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51475,KCLANPO23N000051476,장소,관광지,23412510,삼척맹방해변길,NaN,NaN,60513,일반관광지,5123031022102210009,...,NaN,NaN,NaN,129.230584,37.391000,마사531338,20240108,KT,KC_495_LLR_ATRCTN_2023,20231220
51476,KCLANPO23N000051477,장소,관광지,23412511,영덕블루로드,NaN,NaN,60513,일반관광지,4777031031105810000,...,NaN,NaN,NaN,129.391512,36.358874,마바697196,20240108,KT,KC_495_LLR_ATRCTN_2023,20231220
51477,KCLANPO23N000051478,장소,관광지,23412512,관악별빛산책,NaN,NaN,60523,지역축제,1162010200116420007,...,NaN,NaN,NaN,126.926914,37.483544,다사493428,20240108,KT,KC_495_LLR_ATRCTN_2023,20231220
51478,KCLANPO23N000051479,장소,관광지,23412513,서면빛축제,NaN,NaN,60523,지역축제,2623010300105760001,...,2.623042e+11,신천대로78번길,51,129.057691,35.154344,마라418854,20240108,KT,KC_495_LLR_ATRCTN_2023,20231220


In [11]:
tour_df.columns

Index(['ID', 'LCLAS_NM', 'MLSFC_NM', 'POI_ID', 'POI_NM', 'BHF_NM', 'ASSTN_NM',
       'CL_CD', 'CL_NM', 'PNU', 'CTPRVN_NM', 'SIGNGU_NM', 'LEGALDONG_NM',
       'LI_NM', 'LNBR_NO', 'LEGALDONG_CD', 'ADSTRD_CD', 'RDNMADR_CD',
       'RDNMADR_NM', 'BULD_NO', 'LC_LO', 'LC_LA', 'GID_CD', 'LAST_CHG_DE',
       'ORIGIN_NM', 'FILE_NM', 'BASE_DE'],
      dtype='object')

In [12]:
tour_df['CTPRVN_NM']

,CTPRVN_NM
0,울산광역시
1,전라북도
2,서울특별시
3,서울특별시
4,부산광역시
...,...
51475,강원특별자치도
51476,경상북도
51477,서울특별시
51478,부산광역시


In [13]:
seoul_tour = tour_df[tour_df['CTPRVN_NM'] == '서울특별시']
seoul_tour

,ID,LCLAS_NM,MLSFC_NM,POI_ID,POI_NM,BHF_NM,ASSTN_NM,CL_CD,CL_NM,PNU,...,RDNMADR_CD,RDNMADR_NM,BULD_NO,LC_LO,LC_LA,GID_CD,LAST_CHG_DE,ORIGIN_NM,FILE_NM,BASE_DE
2,KCLANPO23N000000003,장소,관광지,142394,우정여행사,NaN,NaN,60403,항공사/여행사,1114010300100310023,...,1.114041e+11,세종대로20길,15,126.978339,37.567237,다사539521,20240108,KT,KC_495_LLR_ATRCTN_2023,20231220
3,KCLANPO23N000000004,장소,관광지,145432,비산항공여행사,NaN,NaN,60403,항공사/여행사,1165010700100520006,...,1.165031e+11,서초중앙로,225,127.011566,37.501664,다사568448,20240108,KT,KC_495_LLR_ATRCTN_2023,20231220
6,KCLANPO23N000000007,장소,관광지,157191,유타항공여행사,NaN,NaN,60403,항공사/여행사,1165010600100130001,...,1.165042e+11,강남대로101길,17,127.018412,37.516701,다사574464,20240108,KT,KC_495_LLR_ATRCTN_2023,20231220
9,KCLANPO23N000000010,장소,관광지,167131,미르탑항공사,NaN,NaN,60403,항공사/여행사,1111013800100750009,...,1.111031e+11,종로,78,126.986053,37.569904,다사546524,20240108,KT,KC_495_LLR_ATRCTN_2023,20231220
11,KCLANPO23N000000012,장소,관광지,170115,문화관광,NaN,NaN,60403,항공사/여행사,1114011800100630009,...,1.114020e+11,세종대로,3-1,126.972501,37.557685,다사534510,20240108,KT,KC_495_LLR_ATRCTN_2023,20231220
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51442,KCLANPO23N000051443,장소,관광지,23408365,한아국제여행사,NaN,NaN,60403,항공사/여행사,1154510300109350000,...,1.154531e+11,독산로,8,126.904021,37.448504,다사472389,20240108,KT,KC_495_LLR_ATRCTN_2023,20231220
51450,KCLANPO23N000051451,장소,관광지,23409202,스마트락,NaN,NaN,60520,캠핑장,1153010200101970010,...,1.153041e+11,디지털로33길,55,126.893040,37.486756,다사463432,20240108,KT,KC_495_LLR_ATRCTN_2023,20231220
51455,KCLANPO23N000051456,장소,관광지,23409470,강호여행사,NaN,NaN,60403,항공사/여행사,1165010600100390001,...,1.165042e+11,신반포로45길,26,127.018696,37.511717,다사574459,20240108,KT,KC_495_LLR_ATRCTN_2023,20231220
51458,KCLANPO23N000051459,장소,관광지,23410017,바카티오,NaN,NaN,60403,항공사/여행사,1141011200101100038,...,1.141041e+11,연세로2마길,29,126.942005,37.558394,다사507511,20240108,KT,KC_495_LLR_ATRCTN_2023,20231220


In [14]:
tour_df['CL_NM'].unique()

array(['일반관광지', '항공사/여행사', '천연기념물', '관광안내소/매표소', '유명사적/유적지', '영어마을',
       '먹거리/패션거리', '동물원', '테마공원/대형놀이공원', 'N', '비/탑/문/각', '야영장', '식물원',
       '관광농원/허브마을', '팜스테이', '유명관광지', '드라마/영화촬영지', '일반유원지/일반놀이공원', '국보',
       '캠핑장', '폭포/계곡', '서원/향교/서당', '정보화마을', '휴양림/수목원', '지역축제',
       '고택/생가/민속마을', '해수욕장', '보물', '왕릉/고분', '아쿠아리움/대형수족관', '온천지역',
       '궁궐/종묘', '성/성터', '캠핑홀리데이(캠핑)', '글램핑코리아(캠핑)', '잼핑홀리데이(캠핑)'],
      dtype=object)

In [15]:
print(tour_df[tour_df['CL_NM'] == '일반관광지'])

                        ID LCLAS_NM MLSFC_NM    POI_ID     POI_NM   BHF_NM  \
0      KCLANPO23N000000001       장소      관광지     76838      간절곶등대      NaN   
28     KCLANPO23N000000029       장소      관광지    222649       삼청동길      NaN   
35     KCLANPO23N000000036       장소      관광지    248095    선진전위치료기      NaN   
60     KCLANPO23N000000061       장소      관광지    350176       창원의집      NaN   
62     KCLANPO23N000000063       장소      관광지    453527     헤모수불가마      NaN   
...                    ...      ...      ...       ...        ...      ...   
51423  KCLANPO23N000051424       장소      관광지  23406809    경기옛길평해길  제10길솔치길   
51473  KCLANPO23N000051474       장소      관광지  23412507  한탄강물윗길트레킹      NaN   
51474  KCLANPO23N000051475       장소      관광지  23412509    동해망상해변길      NaN   
51475  KCLANPO23N000051476       장소      관광지  23412510    삼척맹방해변길      NaN   
51476  KCLANPO23N000051477       장소      관광지  23412511     영덕블루로드      NaN   

       ASSTN_NM  CL_CD  CL_NM                  PNU  ...    RDNM

In [ ]:
print(tour_df[tour_df['CL_NM'] == 'N'])

                        ID LCLAS_NM MLSFC_NM    POI_ID        POI_NM BHF_NM  \
69     KCLANPO23N000000070       장소      관광지    539117           향나무    NaN   
605    KCLANPO23N000000606       장소      관광지   6974497          구리타워    NaN   
16847  KCLANPO23N000016848       장소      관광지  16925934         일본군관사    NaN   
16887  KCLANPO23N000016888       장소      관광지  16940796    씨없는곶감마을휴양관    NaN   
16916  KCLANPO23N000016917       장소      관광지  16982026         오일장개발    NaN   
...                    ...      ...      ...       ...           ...    ...   
51131  KCLANPO23N000051132       장소      관광지  23322269  외할머니집가는길체험농장    NaN   
51214  KCLANPO23N000051215       장소      관광지  23338249           사물제    NaN   
51215  KCLANPO23N000051216       장소      관광지  23338250           삼성제    NaN   
51216  KCLANPO23N000051217       장소      관광지  23338251           명륜당    NaN   
51322  KCLANPO23N000051323       장소      관광지  23394867         쉬어가삼례    NaN   

       ASSTN_NM  CL_CD CL_NM                  PNU  

In [16]:
seoul_tour = seoul_tour[['POI_NM', 'CL_NM', 'CTPRVN_NM', 'LC_LO', 'LC_LA']]
seoul_tour

,POI_NM,CL_NM,CTPRVN_NM,LC_LO,LC_LA
2,우정여행사,항공사/여행사,서울특별시,126.978339,37.567237
3,비산항공여행사,항공사/여행사,서울특별시,127.011566,37.501664
6,유타항공여행사,항공사/여행사,서울특별시,127.018412,37.516701
9,미르탑항공사,항공사/여행사,서울특별시,126.986053,37.569904
11,문화관광,항공사/여행사,서울특별시,126.972501,37.557685
...,...,...,...,...,...
51442,한아국제여행사,항공사/여행사,서울특별시,126.904021,37.448504
51450,스마트락,캠핑장,서울특별시,126.893040,37.486756
51455,강호여행사,항공사/여행사,서울특별시,127.018696,37.511717
51458,바카티오,항공사/여행사,서울특별시,126.942005,37.558394


In [17]:
# 항공사/여행사 위치는 관광지랑 관련이 적어서 제외
seoul_tour = seoul_tour[seoul_tour['CL_NM'] != '항공사/여행사']
seoul_tour

,POI_NM,CL_NM,CTPRVN_NM,LC_LO,LC_LA
28,삼청동길,일반관광지,서울특별시,126.981658,37.587755
59,서울영어마을,영어마을,서울특별시,127.008515,37.639904
66,어린이대공원매표소,관광안내소/매표소,서울특별시,127.083936,37.551306
67,연세대학교신촌캠퍼스매표소,관광안내소/매표소,서울특별시,126.941825,37.563999
69,향나무,N,서울특별시,126.989473,37.579318
...,...,...,...,...,...
51290,누옥,팜스테이,서울특별시,126.965936,37.579542
51313,무월관,팜스테이,서울특별시,126.983597,37.577241
51333,클래식고택,팜스테이,서울특별시,126.985648,37.580387
51450,스마트락,캠핑장,서울특별시,126.893040,37.486756


In [18]:
# 관광안내소/매표소와 관광지가 중복이 되기 때문에 제외
seoul_tour = seoul_tour[seoul_tour['CL_NM'] != '관광안내소/매표소']
seoul_tour

,POI_NM,CL_NM,CTPRVN_NM,LC_LO,LC_LA
28,삼청동길,일반관광지,서울특별시,126.981658,37.587755
59,서울영어마을,영어마을,서울특별시,127.008515,37.639904
69,향나무,N,서울특별시,126.989473,37.579318
122,남산공원남산케이블카,유명관광지,서울특별시,126.983974,37.556648
136,신당동떡볶이골목,먹거리/패션거리,서울특별시,127.015048,37.563323
...,...,...,...,...,...
51290,누옥,팜스테이,서울특별시,126.965936,37.579542
51313,무월관,팜스테이,서울특별시,126.983597,37.577241
51333,클래식고택,팜스테이,서울특별시,126.985648,37.580387
51450,스마트락,캠핑장,서울특별시,126.893040,37.486756


In [19]:
unique_tour = list(seoul_tour['POI_NM'].unique())
print(unique_tour)

['삼청동길', '서울영어마을', '향나무', '남산공원남산케이블카', '신당동떡볶이골목', '압구정로데오거리', '문정로데오거리', '신림동순대골목', '가리봉로데오거리', '인사동거리', '장충동족발골목', '장통교', '수표교', '새벽다리', '나래교', '맑은내다리', '광통교', '불광동먹자골목', '반포치킨거리', '성신여대패션거리', '삼각지대구탕골목', '자동차용품거리', '패션문화의거리', '종로아구찜골목', '사주카페골목', '신천먹자골목', '방이먹자골목', '방배동카페골목', '신사동먹자골목', '천호동먹자골목', '무교동낙지골목', '북창동먹자골목', '창동로데오거리', '마포돼지갈비골목', '세계음식거리', '빌라촌음식골목', '건대앞먹자골목', '교대곱창골목', '난지한강공원난지캠핑장', '예술거리', '이태원관광특구', '가회동31번지', '가로수길', '남대문시장갈치조림골목', '감성의거리', '고가구골목', '외국인거리', '곱창골목', '극장거리', '기찻길고기골목', '놀이터골목', '대학로주변극장', '대학로거리', '유흥가', '일본거리', '이촌지구', '명동화교거리', '이경직신도비', '중국대사관거리', '덕수궁돌담길', '서울천년타임캡슐', '서울선릉과정릉', '용마공원용마랜드', '청계천', '어린이대공원', '롯데월드', '로데오패션타운', '코엑스아쿠아리움', '63스퀘어아쿠아플라넷', '남산서울타워', '영은문주초', '남산공원남산야외식물원', '노원영어마을', '영어체험마을', '국립산림과학원홍릉수목원', '구파발폭포', '북한산둘레캠핑장', '창덕궁', '경복궁', '경희궁', '운현궁', '창경궁', '광평대군묘역', '서울방이동고분군', '사육신묘', '선릉', '정릉', '정현왕후묘', '헌인릉', '양녕대군묘소', '광화문', '광희문', '낙성대', '남산봉수대', '망원정', '몽촌토성', '백제석촌고분', '백제요지', '보신각', '삼성산성지', '마애여래입상', '새남터순교성지', '서

In [20]:
seoul_tour.isna().sum()

,0
POI_NM,0
CL_NM,0
CTPRVN_NM,0
LC_LO,0
LC_LA,0


In [21]:
seoul_tour.shape

(1601, 5)

### 3. 숙소 데이터 전처리 및 EDA

In [ ]:
print(os.listdir('/content/'))

['.config', '국내_지역별_관광명소데이터.csv', 'drive', '서울시_관광숙박업_인허가_정보.csv', '서울시_정류장마스터_정보.csv', 'sample_data']


In [ ]:
seoul_accom = pd.read_csv('./서울시_관광숙박업_인허가_정보.csv', encoding='cp949')
seoul_accom

,개방자치단체코드,관리번호,인허가일자,인허가취소일자,영업상태코드,영업상태명,상세영업상태코드,상세영업상태명,폐업일자,휴업시작일자,...,의무실유무,안내소유무,기획여행보험시작일자,기획여행보험종료일자,자본금,보험시작일자,보험종료일자,부대시설내역,시설규모,관광숙박업상세명
0,3230000,CDFI2260032019000001,2019-07-25,NaN,1,영업/정상,13,영업중,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3220000,CDFI2260032015000001,2015-02-03,NaN,1,영업/정상,13,영업중,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3010000,CDFI2260032016000008,2016-05-17,NaN,3,폐업,3,폐업,2023-02-28,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3170000,CDFI2260032021000001,2021-04-06,NaN,1,영업/정상,13,영업중,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3010000,CDFI2260032016000006,2016-05-02,NaN,1,영업/정상,13,영업중,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
644,3010000,CDFI2260032018000011,2018-11-20,NaN,1,영업/정상,13,영업중,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
645,3230000,CDFI2260032021000001,2021-06-22,NaN,1,영업/정상,13,영업중,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
646,3240000,CDFI2260032017000001,2017-12-26,NaN,1,영업/정상,13,영업중,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
647,3240000,CDFI2260032014000001,2014-07-24,NaN,1,영업/정상,13,영업중,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
seoul_accom.columns

Index(['개방자치단체코드', '관리번호', '인허가일자', '인허가취소일자', '영업상태코드', '영업상태명', '상세영업상태코드',
       '상세영업상태명', '폐업일자', '휴업시작일자', '휴업종료일자', '재개업일자', '전화번호', '소재지면적',
       '소재지우편번호', '지번주소', '도로명주소', '도로명우편번호', '사업장명', '최종수정일자', '데이터갱신구분',
       '데이터갱신일자', '업태구분명', '좌표정보(X)', '좌표정보(Y)', '문화체육업종명', '문화사업자구분명',
       '지역구분명', '총층수', '주변환경명', '제작취급품목내용', '보험기관명', '건물용도명', '지상층수', '지하층수',
       '객실수', '건축연면적', '영문상호명', '영문상호주소', '선박총톤수', '선박척수', '선박제원', '무대면적',
       '좌석수', '기념품종류', '회의실별동시수용인원', '시설면적', '놀이기구수내역', '놀이시설수', '방송시설유무',
       '발전시설유무', '의무실유무', '안내소유무', '기획여행보험시작일자', '기획여행보험종료일자', '자본금', '보험시작일자',
       '보험종료일자', '부대시설내역', '시설규모', '관광숙박업상세명'],
      dtype='object')

In [ ]:
seoul_accom['상세영업상태명'].unique()

array(['영업중', '폐업', '휴업', '직권말소', '등록취소'], dtype=object)

In [ ]:
seoul_accom = seoul_accom[seoul_accom['상세영업상태명'] == '영업중']
seoul_accom = seoul_accom[['사업장명', '도로명주소', '좌표정보(X)', '좌표정보(Y)']]
seoul_accom

,사업장명,도로명주소,좌표정보(X),좌표정보(Y)
0,호텔 더 캐슬(신천),서울특별시 송파구 올림픽로12길 4-17 (잠실동),207163.343016,445433.784924
1,호텔 데님(Hotel Denim),서울특별시 강남구 논현로 66 (개포동),204053.596301,441557.179656
3,위너스,서울특별시 금천구 가산로3길 103 (독산동),190183.499829,441092.525925
4,코트야드 메리어트 남대문호텔,"서울특별시 중구 남대문로 9 (남대문로4가, 남대문호텔)",197891.564897,450990.593466
5,라마다서울동대문호텔(정광개발주식회사),"서울특별시 중구 동호로 354 (을지로5가, 극동 스타클래스)",200161.192418,451521.415517
...,...,...,...,...
644,토요코인 서울 동대문 Ⅱ,서울특별시 중구 퇴계로 325 (광희동2가),200625.524648,451359.906142
645,호텔 더 캐슬(방이2),서울특별시 송파구 올림픽로32길 7-1 (방이동),209577.698775,445907.772552
646,호텔 감(HOTEL GAM),서울특별시 강동구 천호대로159길 22-36 (천호동),211302.314479,448470.034451
647,하모니관광호텔,서울특별시 강동구 천중로40길 63 (길동),212136.459032,448450.725351


In [ ]:
seoul_accom.isna().sum()

,0
사업장명,0
도로명주소,0
좌표정보(X),2
좌표정보(Y),2


In [ ]:
# 다른 데이터셋들과 비교하기 위해 좌표 정보를 위도, 경도로 변환해주는 함수

from pyproj import CRS, Transformer

def tm2097_to_latlon(x, y):
    """
    중부원점TM (EPSG:2097) 좌표를 위도, 경도로 변환합니다.

    Args:
        x (float): TM2097 x 좌표 (동서 방향)
        y (float): TM2097 y 좌표 (남북 방향)

    Returns:
        tuple: (latitude, longitude)
    """
    # EPSG:2097 (중부원점TM) 좌표계
    crs_tm2097 = CRS.from_epsg(2097)
    # WGS84 좌표계
    crs_wgs84 = CRS.from_epsg(4326)

    # Transformer 객체 생성
    transformer = Transformer.from_crs(crs_tm2097, crs_wgs84, always_xy=True)

    # 좌표 변환
    longitude, latitude = transformer.transform(x, y)
    return latitude, longitude

# 테스트 예제
tm_x = 200000  # 중부원점TM x 좌표
tm_y = 500000  # 중부원점TM y 좌표

latitude, longitude = tm2097_to_latlon(tm_x, tm_y)
print(f"위도: {latitude}, 경도: {longitude}")

위도: 38.002746006367744, 경도: 126.99789352437391


In [ ]:
seoul_accom[['좌표정보(X)', '좌표정보(Y)']].apply(lambda x: tm2097_to_latlon(x['좌표정보(X)'], x['좌표정보(Y)']), axis=1)

,0
0,"(37.511099381990476, 127.07891971732056)"
1,"(37.47618986019083, 127.04372579836088)"
3,"(37.471959134000386, 126.88692676527495)"
4,"(37.56119057589254, 126.97403441984726)"
5,"(37.56597578035972, 126.99972238831533)"
...,...
644,"(37.56452043342592, 127.00497796517418)"
645,"(37.51534826995913, 127.1062334866759)"
646,"(37.53841467455236, 127.12578006840313)"
647,"(37.53823016269024, 127.13521779518449)"


In [ ]:
# apply로 좌표 변환 후 두 개의 컬럼으로 분리
seoul_accom[['위도', '경도']] = seoul_accom.apply(
   lambda row: pd.Series(tm2097_to_latlon(row['좌표정보(X)'], row['좌표정보(Y)'])),
   axis=1
)

seoul_accom

,사업장명,도로명주소,좌표정보(X),좌표정보(Y),위도,경도
0,호텔 더 캐슬(신천),서울특별시 송파구 올림픽로12길 4-17 (잠실동),207163.343016,445433.784924,37.511099,127.078920
1,호텔 데님(Hotel Denim),서울특별시 강남구 논현로 66 (개포동),204053.596301,441557.179656,37.476190,127.043726
3,위너스,서울특별시 금천구 가산로3길 103 (독산동),190183.499829,441092.525925,37.471959,126.886927
4,코트야드 메리어트 남대문호텔,"서울특별시 중구 남대문로 9 (남대문로4가, 남대문호텔)",197891.564897,450990.593466,37.561191,126.974034
5,라마다서울동대문호텔(정광개발주식회사),"서울특별시 중구 동호로 354 (을지로5가, 극동 스타클래스)",200161.192418,451521.415517,37.565976,126.999722
...,...,...,...,...,...,...
644,토요코인 서울 동대문 Ⅱ,서울특별시 중구 퇴계로 325 (광희동2가),200625.524648,451359.906142,37.564520,127.004978
645,호텔 더 캐슬(방이2),서울특별시 송파구 올림픽로32길 7-1 (방이동),209577.698775,445907.772552,37.515348,127.106233
646,호텔 감(HOTEL GAM),서울특별시 강동구 천호대로159길 22-36 (천호동),211302.314479,448470.034451,37.538415,127.125780
647,하모니관광호텔,서울특별시 강동구 천중로40길 63 (길동),212136.459032,448450.725351,37.538230,127.135218


In [ ]:
seoul_accom = seoul_accom.dropna()

### 4. 세가지 데이터 합치기
- seoul_bus: 서울 버스 정류장 정보
- seoul_tour: 서울 관광지 정보
- seoul_accom: 서울 숙박업 정보

In [ ]:
print(seoul_bus.shape)
print(seoul_tour.shape)
print(seoul_accom.shape)

(11217, 7)
(1662, 5)
(468, 6)


In [ ]:
seoul_tour.isna().sum()

,0
POI_NM,0
CL_NM,0
CTPRVN_NM,0
LC_LO,0
LC_LA,0


In [ ]:
print(seoul_bus.dtypes)
print("\n")
print(seoul_tour.dtypes)
print("\n")
print(seoul_accom.dtypes)

정류장_ID               int64
정류장_명칭              object
정류장_유형              object
정류장_번호               int64
위도                 float64
경도                 float64
버스도착정보안내기_설치_여부     object
dtype: object


POI_NM        object
CL_NM         object
CTPRVN_NM     object
LC_LO        float64
LC_LA        float64
dtype: object


사업장명        object
도로명주소       object
좌표정보(X)    float64
좌표정보(Y)    float64
위도         float64
경도         float64
dtype: object


In [ ]:
seoul_bounds = {'lat_min': 37.413294, 'lat_max': 37.715133, 'lon_min': 126.734086, 'lon_max': 127.269311}
print(seoul_bus[(seoul_bus['경도'] < seoul_bounds['lon_min']) |
                (seoul_bus['경도'] > seoul_bounds['lon_max'])])


         정류장_ID 정류장_명칭 정류장_유형  정류장_번호         위도         경도 버스도착정보안내기_설치_여부
3337  108900095  8번지입구   마을버스    9642  37.440965  126.45723           미설치  


In [ ]:
from geopy.distance import geodesic

def calculate_distance(lat1, lon1, lat2, lon2):
    return geodesic((lat1, lon1), (lat2, lon2)).meters


In [ ]:
seoul_tour.head()

,POI_NM,CL_NM,CTPRVN_NM,LC_LO,LC_LA
28,삼청동길,일반관광지,서울특별시,126.981658,37.587755
59,서울영어마을,영어마을,서울특별시,127.008515,37.639904
66,어린이대공원매표소,관광안내소/매표소,서울특별시,127.083936,37.551306
67,연세대학교신촌캠퍼스매표소,관광안내소/매표소,서울특별시,126.941825,37.563999
69,향나무,N,서울특별시,126.989473,37.579318


In [22]:
seoul_tour.rename(columns={'LC_LA': '위도', 'LC_LO': '경도'}, inplace=True)
seoul_tour.rename(columns={'POI_NM': '관광지명'}, inplace=True)

In [30]:
seoul_tour.rename(columns={'CL_NM': '관광지_분류'}, inplace=True)
seoul_tour.rename(columns={'관광지명': 'tour_name'}, inplace=True)

In [31]:
seoul_tour

,tour_name,관광지_분류,CTPRVN_NM,경도,위도
28,삼청동길,일반관광지,서울특별시,126.981658,37.587755
59,서울영어마을,영어마을,서울특별시,127.008515,37.639904
69,향나무,N,서울특별시,126.989473,37.579318
122,남산공원남산케이블카,유명관광지,서울특별시,126.983974,37.556648
136,신당동떡볶이골목,먹거리/패션거리,서울특별시,127.015048,37.563323
...,...,...,...,...,...
51290,누옥,팜스테이,서울특별시,126.965936,37.579542
51313,무월관,팜스테이,서울특별시,126.983597,37.577241
51333,클래식고택,팜스테이,서울특별시,126.985648,37.580387
51450,스마트락,캠핑장,서울특별시,126.893040,37.486756


In [ ]:
seoul_accom.head()

,사업장명,도로명주소,좌표정보(X),좌표정보(Y),위도,경도,distance_to_tour
0,호텔 더 캐슬(신천),서울특별시 송파구 올림픽로12길 4-17 (잠실동),207163.343016,445433.784924,37.511099,127.078920,12093.463070
1,호텔 데님(Hotel Denim),서울특별시 강남구 논현로 66 (개포동),204053.596301,441557.179656,37.476190,127.043726,13543.192183
3,위너스,서울특별시 금천구 가산로3길 103 (독산동),190183.499829,441092.525925,37.471959,126.886927,15338.967114
4,코트야드 메리어트 남대문호텔,"서울특별시 중구 남대문로 9 (남대문로4가, 남대문호텔)",197891.564897,450990.593466,37.561191,126.974034,3024.265283
5,라마다서울동대문호텔(정광개발주식회사),"서울특별시 중구 동호로 354 (을지로5가, 극동 스타클래스)",200161.192418,451521.415517,37.565976,126.999722,2896.394621


In [ ]:
import pandas as pd

# 결과 저장용 리스트
tour_data = []

for _, tour in seoul_tour.iterrows():
    # 관광지 위치
    tour_lat, tour_lon = tour['위도'], tour['경도']  # 현재 행의 위도와 경도 사용

    # 가장 가까운 버스 정류장
    seoul_bus['distance_to_tour'] = seoul_bus.apply(
        lambda x: calculate_distance(tour_lat, tour_lon, x['위도'], x['경도']), axis=1
    )
    nearest_bus = seoul_bus.loc[seoul_bus['distance_to_tour'].idxmin()]

    # 가장 가까운 숙소
    seoul_accom['distance_to_tour'] = seoul_accom.apply(
        lambda x: calculate_distance(tour_lat, tour_lon, x['위도'], x['경도']), axis=1
    )
    nearest_accom = seoul_accom.loc[seoul_accom['distance_to_tour'].idxmin()]

    # 데이터 저장
    tour_data.append({
        'tour_name': tour['관광지명'],  # 관광지 이름
        'nearest_bus_stop': nearest_bus['정류장_명칭'],  # 가장 가까운 버스 정류장 이름
        'nearest_accommodation': nearest_accom['사업장명'],  # 가장 가까운 숙소 이름
        'bus_distance': nearest_bus['distance_to_tour'],  # 버스 정류장과 거리
        'accommodation_distance': nearest_accom['distance_to_tour']  # 숙소와 거리
    })

# 결과 데이터프레임 생성
tour_df = pd.DataFrame(tour_data)
print(tour_df.head())


KeyboardInterrupt: 

In [23]:
import pandas as pd
import numpy as np
from geopy.distance import geodesic
from tqdm import tqdm  # tqdm 라이브러리 추가

# 거리 계산 함수 (벡터화 지원)
def calculate_distances(coord, target_coords):
    return np.array([geodesic(coord, target).meters for target in target_coords])


# 결과 저장용 리스트
tour_data = []

# 관광지 위도/경도 배열
tour_coords = seoul_tour[['위도', '경도']].values

# 버스 정류장 위도/경도 배열
bus_coords = seoul_bus[['위도', '경도']].values

# 숙소 위도/경도 배열
#accom_coords = seoul_accom[['위도', '경도']].values

# tqdm을 사용해 프로그레스 바 추가
for tour in tqdm(seoul_tour.itertuples(), total=len(seoul_tour), desc="Processing Tours"):
    # 관광지 좌표
    tour_coord = (tour.위도, tour.경도)

    # 버스 정류장까지의 거리 계산
    bus_distances = calculate_distances(tour_coord, bus_coords)
    nearest_bus_idx = np.argmin(bus_distances)
    nearest_bus = seoul_bus.iloc[nearest_bus_idx]

    # 숙소까지의 거리 계산
 #   accom_distances = calculate_distances(tour_coord, accom_coords)
 #   nearest_accom_idx = np.argmin(accom_distances)
 #   nearest_accom = seoul_accom.iloc[nearest_accom_idx]

    # 데이터 저장
    tour_data.append({
        'tour_name': tour.관광지명,  # 관광지 이름
        'nearest_bus_stop': nearest_bus['정류장_명칭'],  # 가장 가까운 버스 정류장 이름
    #    'nearest_accommodation': nearest_accom['사업장명'],  # 가장 가까운 숙소 이름
        'bus_distance': bus_distances[nearest_bus_idx],  # 버스 정류장과 거리
    #    'accommodation_distance': accom_distances[nearest_accom_idx]  # 숙소와 거리
    })

# 결과 데이터프레임 생성
tour_df = pd.DataFrame(tour_data)
print(tour_df.head())


Processing Tours: 100%|██████████| 1601/1601 [1:05:44<00:00,  2.46s/it]

    tour_name  nearest_bus_stop  bus_distance
0        삼청동길      삼청공원삼거리.옥호정터     38.025166
1      서울영어마을              모아빌라    197.545979
2         향나무               볼링장    133.892739
3  남산공원남산케이블카    남산케이블카.남산산책로입구    153.510646
4    신당동떡볶이골목  신당동떡볶이타운.중구여성플라자     58.753759


In [25]:
tour_df['관광지_분류'] = seoul_tour['CL_NM']

In [28]:
tour_df = tour_df.drop(columns=['관광지_분류'])

In [32]:
# 'tour_name' 컬럼을 기준으로 병합
tour_df = tour_df.merge(seoul_tour[['tour_name', '관광지_분류']], how='left', on='tour_name')

# 'CL_NM' 컬럼 이름을 '여행지_분류'로 변경
tour_df.rename(columns={'관광지_분류': 'tour_category'}, inplace=True)

# 결과 확인
print(tour_df.head())


    tour_name  nearest_bus_stop  bus_distance tour_category
0        삼청동길      삼청공원삼거리.옥호정터     38.025166         일반관광지
1      서울영어마을              모아빌라    197.545979          영어마을
2         향나무               볼링장    133.892739             N
3  남산공원남산케이블카    남산케이블카.남산산책로입구    153.510646         유명관광지
4    신당동떡볶이골목  신당동떡볶이타운.중구여성플라자     58.753759      먹거리/패션거리


In [33]:
tour_df.head()

,tour_name,nearest_bus_stop,bus_distance,tour_category
0,삼청동길,삼청공원삼거리.옥호정터,38.025166,일반관광지
1,서울영어마을,모아빌라,197.545979,영어마을
2,향나무,볼링장,133.892739,N
3,남산공원남산케이블카,남산케이블카.남산산책로입구,153.510646,유명관광지
4,신당동떡볶이골목,신당동떡볶이타운.중구여성플라자,58.753759,먹거리/패션거리


In [34]:
# CSV 파일로 저장 (파일명은 'tour_accessibility.csv'로 지정)
tour_df.to_csv('tour_accessibility.csv', index=False)

# 저장된 파일 경로 출력
print("CSV 파일이 저장되었습니다: tour_accessibility.csv")

CSV 파일이 저장되었습니다: tour_accessibility.csv


In [38]:
# EXCEL 파일로 저장
tour_df.to_excel('tour_accessibility.xlsx', index=False)

# 저장된 파일 경로 출력
print("excel 파일이 저장되었습니다.")

excel 파일이 저장되었습니다.


In [41]:
len(tour_df['tour_name'].unique())

1447

In [35]:
tour_df['tour_category'].value_counts()

,count
tour_category,
일반관광지,8030
지역축제,401
N,289
먹거리/패션거리,285
팜스테이,128
유명사적/유적지,76
캠핑장,28
비/탑/문/각,24
왕릉/고분,23


In [44]:
print(tour_df[tour_df['tour_category'] == '일반관광지'])

       tour_name nearest_bus_stop  bus_distance tour_category
0           삼청동길     삼청공원삼거리.옥호정터     38.025166         일반관광지
11           장통교          청계1가.광교     74.553336         일반관광지
12           수표교         청계2가.수표교     67.465247         일반관광지
13          새벽다리        청계5가.광장시장     38.348861         일반관광지
14           나래교        청계6가.평화시장     29.797843         일반관광지
...          ...              ...           ...           ...
9412        레드로드        서교푸르지오아파트    143.062944         일반관광지
9435  서울관악산무장애숲길              신소재    270.262366         일반관광지
9436  서울수락산무장애숲길           상원초등학교    137.494601         일반관광지
9437    윤동주시인의언덕     자하문고개.윤동주문학관    148.252455         일반관광지
9438         무무대            수성동계곡    209.154915         일반관광지

[8030 rows x 4 columns]
